In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

/home/dinesh/Documents/vs code/LLM_Fine-tuning/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

# Load variables from .env into the notebook kernel
load_dotenv()

login(token=os.getenv("HF_TOKEN"))


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

lora_path = "/home/dinesh/Downloads/dataset/fine_tuned_lora_model"
base_model = "meta-llama/Llama-2-7b-hf"   # <-- must be the exact base you trained on

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
# If your tokenizer has no pad token (common for Llama)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, lora_path)
model.eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 17.04it/s]
Some parameters are on the meta device because they were offloaded to the cpu.
Some parameters are on the meta device because they were offloaded to the cpu.
/home/dinesh/Documents/vs code/LLM_Fine-tuning/.venv/lib/python3.12/site-packages/peft/peft_model.py:598: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
  

In [10]:
# Input prompt
# ----------------------------
path = "/home/dinesh/Downloads/dataset/IN-Ext/judgement/1953_L_1.txt"
text = open(path, "r", encoding="utf-8").read()

input_text = (
    "### Instruction: Summarize the following legal text.\n\n"
    f"### Input:\n{text.strip()[:10000]}\n\n"
    "### Response:\n"
).strip()

In [11]:
# Tokenize and generate
# ----------------------------
inputs = tokenizer(
    input_text,
    max_length=4096,
    truncation=True,
    return_tensors="pt",
).to(model.device)

with torch.no_grad():
    generated_output = model.generate(
        **inputs,
        max_new_tokens=1000,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False,   # deterministic (set True + temperature/top_p if you want sampling)
    )

generated_text = tokenizer.decode(
    generated_output[0][inputs["input_ids"].shape[-1]:],
    skip_special_tokens=True
)

print("Generated Output:")
print(generated_text)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generated Output:
Summarize the following legal text.

### Input:
one lakshminarayana iyer a hindu brahmin who owned considerable properties in the tirunelveli district died on 13th december 1924 leaving him surviving a widow ranganayaki and a married daughter ramalakshmi.
ramalakshmi had married the plaintiff and had a number of children from him.
they were all alive in december 1924 when lakshminarayana died.
before his death he executed a will on 16th november 1924 the construction of which is in controversy in this appeal.
by this will he gave the following directions after my lifetime you the aforesaid ranganayaki amminal my wife shall till your lifetime enjoy the aforesaid entire properties the outstandings due to me the debts payable by me and the chit amounts payable by me.
after your lifetime ramalakshmi ammal our daughter and wife of rama ayyar avergal of melagaram village and her heirs shall enjoy them with absolute rights and powers of alienation such as gift exchange and s